In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(Seurat)
    library(Signac)
    library(EnsDb.Hsapiens.v86)
    library(dplyr)
    library(ggplot2)
    library(bedr)
    library(SeuratDisk)
    library(scales)
    library(reshape2)
    library(Hmisc)
    library(tidyr)
    library(tidyverse)
    library(crayon)
    library(SeuratData)
    library(readr)
}))


# Definitions etc

In [2]:
set.seed(1234)
options(digits = 2)
stats <- c()


In [3]:
if (!exists("papermill_run")) {
    prj_name <- "Screen1_66guides"
    secondary_a_path <- "S1/analysis/secondary/"
    save_seurat_h5 <- "YES"
    n_libs <- "4"
}


In [4]:
custom_theme <- theme(
  plot.title = element_text(size = 16, hjust = 0.5),
  legend.key.size = unit(0.7, "cm"),
  legend.text = element_text(size = 14)
)


# Load Seurat files

In [5]:
# Load Seurat objects

libs <- paste0("Lib_", 1:as.integer(n_libs))
integrated_h5_path <-
    paste0(
        secondary_a_path,
        "integrated/seurat_objects/integrated_wo_guide_calls"
    )
seurat_list <- list()

for (lib in libs) {
    h5_path <-
        paste0(
            secondary_a_path,
            lib,
            "/seurat_objects/",
            prj_name,
            "_",
            lib,
            ".h5seurat"
        )
    print(h5_path)
    seurat_list[[lib]] <- LoadH5Seurat(h5_path, verbose = FALSE)
    DefaultAssay(seurat_list[[lib]]) <- "SCT"
}


[1] "/home/vsevim/prj/1012-ckd/S2/analysis/secondary/Lib_1/seurat_objects/Screen2_101vecs_Lib_1.h5seurat"


Validating h5Seurat file



[1] "/home/vsevim/prj/1012-ckd/S2/analysis/secondary/Lib_2/seurat_objects/Screen2_101vecs_Lib_2.h5seurat"


Validating h5Seurat file



[1] "/home/vsevim/prj/1012-ckd/S2/analysis/secondary/Lib_3/seurat_objects/Screen2_101vecs_Lib_3.h5seurat"


...

Validating h5Seurat file



[1] "/home/vsevim/prj/1012-ckd/S2/analysis/secondary/Lib_8/seurat_objects/Screen2_101vecs_Lib_8.h5seurat"


Validating h5Seurat file



[1] "/home/vsevim/prj/1012-ckd/S2/analysis/secondary/Lib_9/seurat_objects/Screen2_101vecs_Lib_9.h5seurat"


Validating h5Seurat file



In [6]:
integration_features <- SelectIntegrationFeatures(object.list = seurat_list)


In [7]:
anchors <- FindIntegrationAnchors(
    object.list = seurat_list,
    anchor.features = integration_features,
    verbose = FALSE
)


Warning message in CheckDuplicateCellNames(object.list = object.list):
"Some cell names are duplicated across objects provided. Renaming to enforce unique cell names."


In [8]:
# this command creates an 'integrated' data assay
seurat_combined <- IntegrateData(anchorset = anchors, verbose = FALSE)


# Run SCTransform on the combined

In [9]:
DefaultAssay(seurat_combined) <- "integrated"


In [10]:
seurat_combined <- SCTransform(
    seurat_combined,
    vst.flavor = "v2",
    verbose = FALSE
) %>%
    RunPCA() %>%
    RunUMAP(
        dims = 1:40,
        reduction.name = "umap.rna",
        reduction.key = "rnaUMAP_",
        verbose = FALSE
    )


PC_ 1 
Positive:  WFDC2, ERBB4, KCNIP4, MECOM, MUC1, S100A2, KRT19, KRT7, MAL, AC019197.1 
	   AGR2, TMPRSS4, KITLG, CSGALNACT1, FAM155A, SLPI, LCN2, SLC12A1, ACSL4, LINC01503 
	   BST2, PDE1A, MACC1, TACSTD2, ITGB6, ARHGEF38, CYP1B1, SFRP1, CD9, PLCB1 
Negative:  FN1, CRYAB, CDH6, MT1X, APOE, VCAM1, NCKAP5, SOD2, MALAT1, FGB 
	   CTSB, AC013287.1, RHEX, SERPINA1, TRPM3, G0S2, VGF, SERPINE1, HP, NEAT1 
	   GDA, SLC13A3, ABLIM3, GPC6, SLC2A9, MT1G, SERPINE2, NRG3, HMGA2, S100A1 
PC_ 2 
Positive:  FTL, FTH1, RPS12, S100A6, RPS2, LGALS1, MT1E, RPL10, TMSB10, CRYAB 
	   RPL17, RPS18, RPL41, MT2A, RPL26, RPLP1, RPS27A, VIM, RPL12, RPL7A 
	   RPS3A, RPL11, RPS8, TMSB4X, RPS3, EEF1A1, RPS6, KRT19, RPL13, RPL10A 
Negative:  KCNIP4, ERBB4, MECOM, MALAT1, AC079352.1, NEAT1, LINC01320, PLCB1, PDE4D, ADAMTS9-AS2 
	   AC019197.1, FGF14, FAM155A, TRABD2B, LRP1B, LINC02532, PKHD1, LINC00621, NLGN1, PDE1A 
	   PKP4, EFNA5, NCKAP5, LINC01781, SDK1, UBE2D2, AC019117.1, LINC00511, WFDC2, KCNQ1OT1 
PC_ 3 

### Perform clustering

In [11]:
seurat_combined <- FindNeighbors(
    seurat_combined,
    reduction = "pca", dims = 1:40
)
seurat_combined <- FindClusters(
    seurat_combined,
    resolution = 0.5
)


Computing nearest neighbor graph



Computing SNN



Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 46051
Number of edges: 1540868

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.8669
Number of communities: 16
Elapsed time: 15 seconds


## Add stats slot

In [12]:
df_stats <- NULL
for (i in 1:length(seurat_list)) {
    df_stats <- rbind(df_stats, as.data.frame(seurat_list[[i]]@misc$stats))
}

Misc(seurat_combined, slot = "stats") <- df_stats


### Save H5

In [13]:
SaveH5Seurat(
    seurat_combined,
    integrated_h5_path,
    verbose = FALSE,
    overwrite = T
)


Warning message:
"Overwriting previous file /home/vsevim/prj/1012-ckd/S2/analysis/secondary/integrated/seurat_objects/integrated_wo_guide_calls.h5seurat"
Creating h5Seurat file for version 3.1.5.9900

